# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

file_path_list

/home/workspace


['/home/workspace/event_data/2018-11-27-events.csv',
 '/home/workspace/event_data/2018-11-04-events.csv',
 '/home/workspace/event_data/2018-11-07-events.csv',
 '/home/workspace/event_data/2018-11-09-events.csv',
 '/home/workspace/event_data/2018-11-19-events.csv',
 '/home/workspace/event_data/2018-11-05-events.csv',
 '/home/workspace/event_data/2018-11-22-events.csv',
 '/home/workspace/event_data/2018-11-16-events.csv',
 '/home/workspace/event_data/2018-11-26-events.csv',
 '/home/workspace/event_data/2018-11-24-events.csv',
 '/home/workspace/event_data/2018-11-29-events.csv',
 '/home/workspace/event_data/2018-11-15-events.csv',
 '/home/workspace/event_data/2018-11-20-events.csv',
 '/home/workspace/event_data/2018-11-06-events.csv',
 '/home/workspace/event_data/2018-11-18-events.csv',
 '/home/workspace/event_data/2018-11-21-events.csv',
 '/home/workspace/event_data/2018-11-10-events.csv',
 '/home/workspace/event_data/2018-11-23-events.csv',
 '/home/workspace/event_data/2018-11-02-events

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [2]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [3]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [4]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Mission One:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
- 1. There should be a unique key to separate each row of this table, according to the where clause, I choose (session_id + iteminsession) as primary key -- don't need to care about the users' infomation
    - partition_key: session_id
    - clustering_columns: iteminsession

In [16]:
## drop table if exists
try:
    session.execute("drop table if exists song_info_by_session")
except Exception as e:
    print(e)

## creata table 
query = "CREATE TABLE IF NOT EXISTS song_info_by_session "
query = query + "(session_id int, iteminsession int, artist text, song_title text, song_length float, PRIMARY KEY (session_id, iteminsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [17]:
# load the data into databases
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "insert into song_info_by_session (session_id, iteminsession, artist, song_title, song_length) "
        query = query + "values (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [18]:
# select
res1 = "select artist, song_title, song_length from song_info_by_session where session_id=338 and iteminsession=4"
try:
    rows = session.execute(res1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Mission two: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
- This mission has include the information of user's, so I need to put the user_id into the primary key
    - partition_key : user_id + session_id
    - clustering columns: iteminsession

In [19]:
# drop table if exists
try:
    session.execute("drop table if exists songplay_info_by_user")
except Exception as e:
    print(e)   

# creata table
query2 = "CREATE TABLE IF NOT EXISTS songplay_info_by_user "
query2 = query2 + "(user_id int, session_id int, iteminsession int, artist text, song text, user text, PRIMARY KEY ((user_id, session_id), iteminsession))"
try:
    session.execute(query2)
except Exception as e:
    print(e)     

In [20]:
# load the data into databases
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "insert into songplay_info_by_user (user_id, session_id, iteminsession, artist, song, user) "
        query = query + "values (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], (line[1]+' '+line[4])))

In [21]:
# select
res2 = "select artist, song, user from songplay_info_by_user where user_id=10 and session_id=182"
try:
    rows = session.execute(res2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.user)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Mission three:  Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
- This mission focuses on the song_title, so I have to use song to partition
    - partition_key : song
    - clustering columns : session_id, iteminsession

In [22]:
# drop table if exists
try:
    session.execute("drop table if exists user_info_by_song")
except Exception as e:
    print(e) 


# create table
query3 = "CREATE TABLE IF NOT EXISTS user_info_by_song "
query3 = query3 + "(song text, user_id int, user text, PRIMARY KEY (song, user_id))"
try:
    session.execute(query3)
except Exception as e:
    print(e)                  

In [23]:
# load the data into databases
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "insert into user_info_by_song (song, user_id, user) "
        query = query + "values (%s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), (line[1]+' '+ line[4])))

In [24]:
# select 
res3 = "select user from user_info_by_song where song = 'All Hands Against His Own'"
try:
    rows = session.execute(res3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [4]:
## Drop the table before closing out the sessions

In [51]:
session.execute("drop table if exists song_info_by_session")
session.execute("drop table if exists songplay_info_by_user")
session.execute("drop table if exists user_info_by_song")

### Close the session and cluster connection¶

In [52]:
session.shutdown()
cluster.shutdown()